In [ ]:
from numpy.random import seed
seed(1)
import  keras
import numpy as np
import json
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm_notebook
import torch
import torch.nn as nn
import torch.optim as optim
import time


In [ ]:
!pip install transformers

In [ ]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

In [ ]:
cd drive/MyDrive/for-github/KhasiRobertaPOS

/content/drive/MyDrive/for-github/KhasiRobertaPOS


In [ ]:
tokenizer = BertTokenizer.from_pretrained('RoPOS_Wordpiecetok1')
from sklearn.model_selection import train_test_split

In [ ]:
!python src/metrics.py
from src.metrics import categorical_accuracy,epoch_time,format_time

In [ ]:

!python ./src/batch.py
from src.batch import make_batches,  make_test_batches

#!python src/training.py
from src.training import train,evaluate, test


!python src/utils.py
from src.utils import *

In [ ]:
Y=[]
X=[]
with open('data/tags.txt', 'r',encoding='utf-8') as fp:
    for line in fp:
      wordlst=[]
      for word in line.split():
         wordlst.append(word.lower())
      if len(wordlst)>0:
        Y.append(wordlst)
with open('data/text.txt', 'r',encoding='utf-8') as fp:
    for line in fp:
      wordlst=[]
      for word in line.split():
         wordlst.append(word.lower())
      if len(wordlst)>0:   #print(wordlst)
        X.append(wordlst)

In [ ]:
TEST_SIZE = 0.1
train_text, test_text, train_label, test_label = train_test_split(X, Y, test_size=TEST_SIZE, random_state=4)
VALID_SIZE = 0.2
train_text, valid_text, train_label, valid_label = train_test_split(train_text, train_label, test_size=VALID_SIZE, random_state=4)


In [ ]:
with open('tagsconfig.json','r') as f:#load the saved json string tokenizer config
  con=json.load(f)
tag_tokenizer=keras.preprocessing.text.tokenizer_from_json(con)


In [ ]:
class InputExample(object):
    def __init__(self, text_a, label=None):
        self.text_a = text_a
        self.label = label

def create_tokenizer():
    tokenizer = BertTokenizer.from_pretrained('RoPOS_Wordpiecetok1')
    return tokenizer

def convert_single_example(tokenizer, example, max_seq_length=512,max_id_length=200):
    tokens_a = example.text_a
    labels_a= example.label
    if len(tokens_a) > max_seq_length-2:
        tokens_a = tokens_a[0 : (max_seq_length-2)]
        labels_a =labels_a[0 : (max_seq_length-2)]

    orig_to_tok_map = []
    tokens = []
    labels=[]

    tokens.append("[CLS]")
    labels.append("<pad>")

    orig_to_tok_map.append(len(tokens)-1)
    #print(len(tokens_a))
    for (i,token) in enumerate(tokens_a):
        orig_to_tok_map.append(len(tokens))#keep first piece
        sub_toks=tokenizer.tokenize(token)
        #print(sub_toks)
        tokens.extend(sub_toks)
        sub_labs=[labels_a[i]]*len(sub_toks)#repeat same labels for subtoks
        labels.extend(sub_labs)

    tokens.append("[SEP]")
    labels.append("<pad>")
    # print('label:',len(labels),'\t','toks:',len(tokens))
    orig_to_tok_map.append(len(tokens)-1)
    input_ids = tokenizer.convert_tokens_to_ids([tok for tok  in tokens])
    label_ids = []
    label_ids.extend([tag_tokenizer.word_index[label] for label in labels])

    if len(input_ids)>max_id_length:
        input_ids=input_ids[0 : max_id_length]
        label_ids=label_ids[0 : max_id_length]
    input_mask = [1] * len(input_ids)
    # Zero-pad up to the sequence length.
    while len(input_ids) < max_id_length:
        input_ids.append(0)
        input_mask.append(0)
        label_ids.append(0)


    return input_ids, input_mask, orig_to_tok_map,label_ids

def convert_examples_to_features(tokenizer, examples, max_seq_length=256):
    input_ids, input_masks, orig_to_tok_maps, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask,orig_to_tok_map, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        labels.append(label)

    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(labels),
    )

def convert_text_to_examples(texts, labels):
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample( text_a=text, label=label)
        )
    return InputExamples

In [ ]:
max_id_length=250
MAX_SEQUENCE_LENGTH = 200
tokenizer = create_tokenizer()
n_tags=len(tag_tokenizer.word_index)

In [ ]:
InputExamples = []
for text, label in zip(test_text[0:1], test_label[0:1]):
    InputExamples.append(
        InputExample( text_a=text, label=label)
)

In [ ]:
#Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
valid_examples = convert_text_to_examples(valid_text, valid_label)
test_examples = convert_text_to_examples(test_text, test_label) #uncomment this for test


In [ ]:
# Convert to features
(train_input_ids, train_input_masks,  train_labels_ids
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)
(valid_input_ids, valid_input_masks,  valid_labels_ids
) = convert_examples_to_features(tokenizer, valid_examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)
# (test_input_ids, test_input_masks,  test_labels_ids
# ) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=MAX_SEQUENCE_LENGTH+2)#uncomment for test

In [ ]:
class RoPOStagger(nn.Module):
    def __init__(self,Rob_lang,output_dim, dropout):
        super().__init__()
        self.Rob_lang = Rob_lang
        embedding_dim = 768
        self.fc = nn.Linear(embedding_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        out=self.Rob_lang(text)
        # print(len(out))  # 3
        hidden_states = out[1]
        embedding_output = hidden_states[0]
        embedded = self.dropout(embedding_output)
        predictions = self.fc(self.dropout(embedded))

        predictions = self.fc(embedded)
       # print(predictions.shape)
        return predictions


In [ ]:
from transformers import RobertaForMaskedLM
Rob_lang = RobertaForMaskedLM.from_pretrained('KhasiPretrained')

In [ ]:
OutputDim = 66#len tags
Dropout = .5
model = RoPOStagger(Rob_lang,OutputDim,Dropout)

In [ ]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
learn_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr = learn_rate)
TAG_PAD_IDX = 0
criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
criterion = criterion.to(device)

In [ ]:
import time
epochs =50

best_valid_acc=0
# (batch_inputs,batch_tags) = make_batches(train_input_ids,train_labels_ids,8)
(batch_val_inputs,batch_val_tags) = make_batches(valid_input_ids,valid_labels_ids,8)

for epoch in range(epochs):
    (batch_inputs,batch_tags) = make_batches(train_input_ids,train_labels_ids,8)
    start_time = time.time()

    train_loss, train_acc = train(batch_inputs,batch_tags,model, optimizer, criterion, TAG_PAD_IDX,device)
    valid_loss, valid_acc = evaluate(batch_val_inputs,batch_val_tags,model,  criterion, TAG_PAD_IDX,device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)


    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'bestmodel.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
#torch.save(model.state_dict(),'model.pt')



**Testing**

In [ ]:
model.load_state_dict(torch.load('model_RoPOStok1_val_4.pt'))

In [ ]:
(batch_test_inputs,batch_test_tags) = make_test_batches(test_input_ids,test_labels_ids,315)
test_loss, test_acc ,ypred= test(model,batch_test_inputs,batch_test_tags, criterion, TAG_PAD_IDX,device)
print(test_loss ,'\t', test_acc)


**Inferencing**

In [ ]:
id2tag={id:t for t,id in tag_tokenizer.word_index.items()}


In [ ]:
fout=open('testv2pos.txt', 'a+',encoding='utf-8')
with open('data/69.txt', 'r',encoding='utf-8') as fp:
    for line in fp:
       if len(line)>0:
            retags=tag_sentence(tokenizer,line,model,id2tag,device)
            wordlst=[word.lower() for word in line.split()]
            #print(retags)
            tagged_sent=join(retags,wordlst)
            #print(tagged_sent)
            fout.write(tagged_sent + '\n')
fp.close()
fout.close()